In [1]:
import numpy as np
import newbridge as nb
import parameters as prm
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
# load data
import pickle
with open('./data/noise_2.pkl','rb') as f:
    xout, tout, x_without_noise, euler_param, sim_param = pickle.load(f)

In [4]:
print(xout.shape)
print(tout.shape)

(10, 1001, 4)
(10, 1001)


In [5]:
def H(degree, x):
    switcher = {
        0: 0.63161877774606470129,
        1: 0.63161877774606470129 * x,
        2: 0.44662192086900116570 * (np.power(x, 2) - 1),
        3: 0.25785728623970555997 * (np.power(x, 3) - 3 * x),
        4: 0.12892864311985277998 * (np.power(x, 4) - 6 * np.power(x, 2) + 3),
    }
    return switcher.get(degree, "Polynomial degree exceeded")

def hermite_basis(x):
#     y = np.zeros((x.shape[0], x.shape[1]))
    index = 0

    for d in range(0, 7):
        for i in range(0, d + 1):
            for j in range(0, d + 1):
                if (i + j == d):
#                                     print(d, i, j, k, l, m, n, index)
#                                     y[:, index] = H(i, x[:, 0]) * H(j, x[:, 1]) * H(k, x[:, 2]) * H(l, x[:, 3]) * H(m, x[:, 4]) * H(n, x[:, 5])
                    index += 1

    print(index)
#     return y

In [6]:
x = np.array([[1., 2., 3., 4., 5., 6.], [2., 4., 6., 8., 9., 10.]])
y = hermite_basis(x)

28


In [7]:
import numpy as np
def choose(degree, dim):
    return np.math.factorial(degree) / (np.math.factorial(dim) * np.math.factorial(degree - dim))

degree = 4
dim = 4
dof = int(choose(degree + dim - 1, dim))
print(dof)

35


In [8]:
def H(degree, x):
    switcher = {
        0: 0.63161877774606470129,
        1: 0.63161877774606470129 * x,
        2: 0.44662192086900116570 * (np.power(x, 2) - 1),
        3: 0.25785728623970555997 * (np.power(x, 3) - 3 * x),
        4: 0.12892864311985277998 * (np.power(x, 4) - 6 * np.power(x, 2) + 3),
    }
    return switcher.get(degree, "Polynomial degree exceeded")

def hermite_basis(x):
    y = np.zeros((x.shape[0], prm.dof))
    index = 0

    for d in range(0, prm.num_hermite_terms):
        for i in range(0, d + 1):
            for j in range(0, d + 1):
                for k in range(0, d + 1):
                    for l in range(0, d + 1):
                        if (i + j + k + l == d):
                            # print("d", d, "i", i, "j", j, "k", k, "l", l, "index", index)
                            y[:, index] = H(i, x[:, 0]) * H(j, x[:, 1]) * H(k, x[:, 2]) * H(l, x[:, 3])
                            index += 1

    return y

def index_mapping():
    index = 0
    index_map = {}

    for d in range(0, prm.num_hermite_terms):
        for i in range(0, d + 1):
            for j in range(0, d + 1):
                for k in range(0, d + 1):
                    for l in range(0, d + 1):
                        if (i + j + k + l == d):
                            index_set = (i, j, k, l)
                            index_map[index_set] = index
                            index += 1

    return index_map

def hermite_to_ordinary(theta):
    transformation = np.zeros((prm.dof, prm.dof))
    y = np.zeros((x.shape[0], prm.dof))
    index_map = index_mapping()
    index = 0
    
    mat = np.zeros((prm.num_hermite_terms, prm.num_hermite_terms))
    mat[0, 0] = H(0, 1)
    mat[1, 1] = H(1, 1)
    mat[2, 2] = H(2, 1)
    mat[2, 0] = -mat[2, 2]
    mat[3, 3] = H(3, 1)
    mat[3, 1] = -3 * mat[3, 3]

    for d in range(0, prm.num_hermite_terms):
        for i in range(0, d + 1):
            for j in range(0, d + 1):
                for k in range(0, d + 1):
                    for l in range(0, d + 1):
                        if (i + j + k + l == d):
                            if (i >= 2):
                                new_index_set = (i - 2, j, k, l)
                                new_index = index_map[new_index_set]
                                transformation[new_index, index] = mat[i, i - 2] * mat[j, j] * mat[k, k] * mat[l, l]
                            if (j >= 2):
                                new_index_set = (i, j - 2, k, l)
                                new_index = index_map[new_index_set]
                                transformation[new_index, index] = mat[i, i] * mat[j, j - 2] * mat[k, k] * mat[l, l]
                            if (k >= 2):
                                new_index_set = (i, j, k - 2, l)
                                new_index = index_map[new_index_set]
                                transformation[new_index, index] = mat[i, i] * mat[j, j] * mat[k, k - 2] * mat[l, l]
                            if (l >= 2):
                                new_index_set = (i, j, k, l - 2)
                                new_index = index_map[new_index_set]
                                transformation[new_index, index] = mat[i, i] * mat[j, j] * mat[k, k] * mat[l, l - 2]
                            
                            # assuming max polynomial degree = 3, then cases would be 
                            # (3, 0, 0, 0) or (2, 1, 0, 0) or (1, 1, 1, 0) or (0, 0, 0, 0)
                            # i.e., at max 1 component has 2 parts
                            # Note: degree 4 is much more complicated because it has 
                            # (4, 0, 0, 0), (2, 2, 0, 0) and (3, 1, 0, 0)
                            transformation[index, index] = mat[i, i] * mat[j, j] * mat[k, k] * mat[l, l]
                            index += 1
                            
    transformed_theta = np.matmul(transformation, np.transpose(theta))
    return np.transpose(transformed_theta)

In [9]:
x = np.array([[1., 2., 3., 4.], [2., 4., 6., 8.]])
theta = hermite_basis(x)
new_theta = hermite_to_ordinary(theta)

In [10]:
print(np.transpose(theta))
print(theta.shape)
print(np.transpose(new_theta))
print(new_theta.shape)

[[  0.15915494   0.15915494]
 [  0.63661977   1.27323954]
 [  0.47746483   0.95492966]
 [  0.31830989   0.63661977]
 [  0.15915494   0.31830989]
 [  1.68809309   7.08999099]
 [  1.90985932   7.63943727]
 [  0.90031632   3.93888388]
 [  1.27323954   5.09295818]
 [  0.95492966   3.81971863]
 [  0.33761862   1.68809309]
 [  0.63661977   2.54647909]
 [  0.47746483   1.90985932]
 [  0.31830989   1.27323954]
 [  0.           0.33761862]
 [  3.37868614  31.70766992]
 [  5.06427928  42.53994594]
 [  3.60126526  31.51107107]
 [  1.1695452   12.86499722]
 [  3.37618619  28.35996396]
 [  3.81971863  30.55774907]
 [  1.80063263  15.75553553]
 [  1.35047447  13.50474474]
 [  1.01285586  10.12855856]
 [  0.12994947   3.37868614]
 [  1.68809309  14.17998198]
 [  1.90985932  15.27887454]
 [  0.90031632   7.87776777]
 [  1.27323954  10.18591636]
 [  0.95492966   7.63943727]
 [  0.33761862   3.37618619]
 [  0.           2.70094895]
 [  0.           2.02571171]
 [  0.           1.35047447]
 [ -0.12994947

In [11]:
index = index_mapping()

In [12]:
print(index)

{(0, 0, 0, 0): 0, (0, 0, 0, 1): 1, (0, 0, 1, 0): 2, (0, 1, 0, 0): 3, (1, 0, 0, 0): 4, (0, 0, 0, 2): 5, (0, 0, 1, 1): 6, (0, 0, 2, 0): 7, (0, 1, 0, 1): 8, (0, 1, 1, 0): 9, (0, 2, 0, 0): 10, (1, 0, 0, 1): 11, (1, 0, 1, 0): 12, (1, 1, 0, 0): 13, (2, 0, 0, 0): 14, (0, 0, 0, 3): 15, (0, 0, 1, 2): 16, (0, 0, 2, 1): 17, (0, 0, 3, 0): 18, (0, 1, 0, 2): 19, (0, 1, 1, 1): 20, (0, 1, 2, 0): 21, (0, 2, 0, 1): 22, (0, 2, 1, 0): 23, (0, 3, 0, 0): 24, (1, 0, 0, 2): 25, (1, 0, 1, 1): 26, (1, 0, 2, 0): 27, (1, 1, 0, 1): 28, (1, 1, 1, 0): 29, (1, 2, 0, 0): 30, (2, 0, 0, 1): 31, (2, 0, 1, 0): 32, (2, 1, 0, 0): 33, (3, 0, 0, 0): 34}
